In [1]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--workers',type=int,default=8,help='Number of workers for dataloader')
parser.add_argument('batch_size',type=int,default=128)
parser.add_argument('n_class',type=int,default=27,help='Number of styles')
parser.add_argument('n_class',type=int,default=27,help='Number of styles')
parser.add_argument('image_size',type=int,default=64,help='Size of traning data')
parser.add_argument('nc',type=int,default=3,help='Number of channels')
parser.add_argument('nz',type=int,default=150,help='z latent vector')
parser.add_argument('G_h_size',type=int,default=64,help='Size of feature maps in generator')
parser.add_argument('D_h_size',type=int,default=32,help='Size of feature maps in discriminator')
parser.add_argument('num_epochs',type=int,default=75)
parser.add_argument('lr',type=float,default=0.0001)
parser.add_argument('beta1 = 0.5',type=float,default=0.5,help='beta1 for adam')
parser.add_argument('ngpu',type=int,default=1)

_StoreAction(option_strings=[], dest='ngpu', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help=None, metavar=None)

In [2]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np

In [3]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') !=-1:
        nn.init.normal_(m.weights.data,0.0,0.02)
    elif classname.find('BatchNorm') !=-1:
        nn.init.normal_(m.weight.data,1.0,0.02)
        nn.init.constant_(m.bias.data,0)

def gaussian(ins, mean, stddev):
    noise = Variable(ins.data.new(ins.size()).normal_(mean, stddev))
    return ins + noise

In [ ]:
# DCGAN generator
class DCGAN_G(nn.Module):
    def __init__(self,ngpu):
        super(DCGAN_G,self).__init__()
        self.label_emb = nn.Embedding(n_class,n_class)
        self.ngpu = ngpu
        main = torch.nn.Sequential()
        
        
        # We need to know how many layers we will use at the beginning
        mult = param.image_size // 8
        
        
        ### Start block
        # Z_size random numbers
        main.add_module('Start-ConvTranspose2d',torch.nn.ConvTranspose2d(param.nz+n_class,param.G_h_size*mult,kernel_size=4,stride=1,padding=0,Bias=False))
        if param.SELU:
            main.add_module('Start-SELU',torch.nn.SELU(inplace=True))
        else:
            main.add_module('Start-BatchNorm2d',torch.nn.Batch2d(param.G_h_size*mult))
            main.add_module('Start-ReLU',torch.nn.ReLU())
        # Size = (G_h_size * mult)x 4 x4
        
        ### Middle block (Done until we reach  ? x image_size/2 x image_size/2)
        i=1
        while mult>1:
            main.add_module('Middle-ConvTranspose2d [%d]'%i, torch.nn.ConvTranspose2d(param.G_h_size*mult,param.G_h_size*(mult//2),kernel_size=4,stride=2,padding=1,bias=False))
            if param.SELU:
                main.add_module('Middle-SELU [%d]'%i,torch.nn.SELU(inplace=True))
            else:
                main.add_module('Middle-BatchNorm2d [%d]'%i,torch.nn.BatchNorm2d(param.G_h_size*(mult//2)))
                main.add_module('Middle_ReLU [$d]'%i,torch.nn.ReLU(inplace=True))
            # Size = (G_h_size * (mult/(2*i)))x8x8
            mult = mult // 2
            i+=1
        
        ### End block
        # Size = G_h_size/2 x image_size/2
        main.add_module('End-ConvTransposed2d', torch.nn.ConvTranspose2d(param.G_h_size,param.n_colors,kernel_size=4,stride=2,padding=1,bias=False))
        main.add_module('End-Tanh',torch.nn.Tanh())
        # Size = n_colors x image_size x image_size
        self.main = main
    
    def forward(self,input,labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels).unsqueeze(2).unsqueeze(3),input),1)
        if isinstance(gen_input.data,torch.cuda.FloatTensor) and param.n_gpu>1:
            output = torch.nn.parallel.data_parallel(self.main,gen_input,range(param.n_gpu))
        else:
            output = self.main(gen_input)
        output = img.view(img.size(0),*(param.n_colors,image_size,image_size))
        return output

In [ ]:
# DCGAN discriminator ( using somewhat the reverse of the generator)
class DCGAN_D(torch.nn.Module):
    def __init__(self,ngpu):
        super(DCGAN_D,self).__init__()
        main = torch.nn.Sequential()
        self.ngpu = ngpu
        self.label_emb = nn.Embedding(n_class,param.D_h_size*16*4)
        
        ### start block
        # size = n_colors x image_size x image_size
        main.add_module('Start-conv2d',torch.nn.Conv2d(param.n_colors,param.D_h_size,kernel=4,stride=2,padding=1,bias=False))
        if param.SELU:
            main.add_module('Start-SELU',torch.nn.SELU(inplace=True))
        else:
            main.add_module('Start-LeakyReLU',torch.nn.LeakyReLU(0.2,inplace=True))
        image_size_new = para.image_size // 2
        # Size = D_h_size x image_size/2 x image_size/2
        
        ### Middle block (Done until we reach ? x 4 x4)
        mult = 1
        i=0
        while image_size_new>4:
            main.add_module('Middle-Conv2d [%d]'%i,torch.nn.Conv2d(param.D_h_size*mult,param.D_h_size*(2*mult),kernel_size=4,stride=2,padding=1,biase=False))
            if param.SELU:
                main.add_module('Middle-SELU [%d]'%i,torch.nn.SELU(inplace=True))
            else:
                main.add_module('Middle-BatchNorm2d [%d]'%i,torch.nn.BatchNorm2d(param.D_h_size*(2*mult)))
                main.add_module('Middle-LeakyReLU [%d]'%i,torch.nn.LeakyReLU(0.2,inplace=True))
            # Size = (D_h_size*(2*i)) x image_size/(2*i) x image_size/(2*i)
            image_size_new = image_size_new // 2
            mult *=2
            i +=1
            
            ## End Block
            # size = (D_h_size*mult) x 4 x4
            main.add_module('End-Conv2d',torch.nn.Conv2d(param.D_h_size*mult,param.D_h_size*16,kernel_size=4,stride=1,padding=0,bias=False))
            if param.SELU:
                main.add_module('Middle-SELU [%d]'%i,torch.nn.SELU(inplace=True))
            else:
                main.add_module('Middle-BatchNorm2d [%d]'%i,torch.nn.BatchNorm2d(param.D_h_size*(2*mult)))
                main.add_module('Middle-LeakyReLU [%d]'%i,torch.nn.LeakyReLU(0.2,inplace=True))
            main.add_module('flatten',torch.nn.Flatten())
#             main.add_module('Sigmoid',torch.nn.Sigmoid())
            
            # size = 1 x1 x1 (Is a real cat or not)
            self.main = main
            self.linear = torch.Sequential(
                torch.nn.Linear(param.D_h_size*16*4*2,param.D_h_size*16),
                torch.nn.SELU(inplace=True) if param.SELU else torch.nn.LeakyReLU(0.2,inplace=True),
                torch.nn.Linear(param.D_h_size*16,1),
                torch.nn.Sigmoid()
            )
            
        def forward(self,input,labels):
            linear_input = torch.cat((self.label_emb(labels),disct_out,1))
            if isinstance(input.data,torch.cuda.FloatTensor) and param.n_gpu>1:
                output = torch.nn.parallel.data_parallel(self.main,input,range(param.n_gpu))
            else:
                output = self.main(input)
            
            linear_input = torch.cat((self.label_emb(labels),disct_out,1))
            if isinstance(input.data,torch.cuda.FloatTensor) and param.n_gpu>1:
                output = torch.nn.parallel.data_parallel(self.linear,linear_input.squeeze(),range(param.n_gpu))
            else:
                output = self.linear(linear_input.squeeze())
            
            # Convert from 1 x 1 x 1 to 1 so that we can compare to given label (cat or not?)
            return output.unsqueeze(2).unsquee(3)


In [4]:
manualSeed = 999
print('Random Seed: ',manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

dataloader = get_dataset()
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis('off')
plt.title('training Images')
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64],padding=2,normilize=True).cpu(),(1,2,0)))

Random Seed:  999


NameError: name 'get_dataset' is not defined

In [5]:
netG = DCGAN_G(param.n_gpu).to(device)
netG.apply(weights_init)
print(netG)

netD = DCGAN_D(param.n_gpu).to(device)
netD.apply(weights_init)
print(netD)

NameError: name 'DCGAN_G' is not defined

In [6]:
criterion = nn.BCELoss()
example_size = 4
fixed_noise = torch.randn(n_class*example,param.nz,1,1,device=device)
fixed_label = torch.tensor(list(np.repeat([i for i in range(n_class)],example_size))).type(torch.LongTensor).to(device)

real_label = 1
fake_label = 0

optimizerD = optim.Adam(netD.parameters(),lr=param.lr,betas=(param.beta1,0.999))
optimizerG = optim.Adam(netG.parameters(),lr=param.lr,betas=(param.beta1,0.999))

from torchvision.utils import save_image
img_save_path = 'images'
os.makedirs(img_save_path,exist_ok=True)

# Training Loop
img_List = []
G_losses = []
D_losses =[]
iters = 0

print('starting Training Loop')
for epoch in range(num_epochs):
    for i,(data,real_style_labels) in enumerate(dataloader,0):
        netD.zero_grad()
        
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size),real_label,dtype=torch.float,device=device)
        
        real_style_labels = real_style_labels.to(device)
        fake_style_labels = torch.tensor(np.random.choice(n_class,size=b_size)).type(torch.LongTensor).to(device)
        
        # Forward pass real batch
        output = netD(gaussian(real_cpu,mean=0,stddev=0.5*0.01**(param.epoch/num_epochs)),real_style_labels).view(-1)
        
        # Calc loss on real
        errD_real = criterion(output,label)
        errD_real.backward()
        D_x = output.mean().item()
        
        # Train with all-fake batch
        # Generate batch of laten vector
        noise =torch.randn(b_size,nz,1,1,device=device)
        # Generator fake image with C
        fake = netG(noise,fake_style_labels)
        label.fill_(fake_label)
        
        # classify all fake batch with D
        output=netD(fake.detach(),fake_style_labels).view(-1)
        # Calc D's loss on fake batch
        errD_fake = criterion(output,label)
        # Calculate gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()
        
        # train G
        netG.zero_grad()
        label.fill_(real_label) # fake labels are real for generator cost
        # Since we just updated D, perform another pass of fake batch
        output = netD(fake,fake_style_label).view(-1)
        # Calculate loss for G
        errG = criterion(output,label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # update G
        optimizerG.step()
        
        if i%200==0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        G_losses.append(errG.item())
        D_losses.append(errG.item())
        
        # check how the generaot is doing by saving G's output on fixed noise
        if (i == len(dataloader)-1):
            with torch.no_grad():
                fake = netG(fixed_noise,fixed_label).detach().cpu()
            img_list.append(vutils.make_grid(fake,nrow=example_size,padding=2,normalize=True))
            save_image(fake.data,img_save_path+'/%d-%d.png'%(epoch,iters),nrow=example_size,normalize=True)
        iters+=1

NameError: name 'n_class' is not defined